In [56]:
import pandas as pd
import sqlalchemy as sa
import urllib

params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()

In [57]:
disease_id_df = pd.read_sql(sa.text('SELECT $node_id, disgenet_id AS dst FROM disease_nodes'), cnxn)
disease_id_df

,$node_id_9F94EAB3BA4046F396391C955D314E4B,dst
0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
1,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
2,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
3,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
4,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
...,...,...
413277,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0020681
413278,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0020681
413279,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C1262019
413280,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C1262019


In [58]:
gene_id_df = pd.read_sql(sa.text('SELECT $node_id, uniprotkb_id as src from gene_nodes'), cnxn)
gene_id_df

,$node_id_70F4A652A18D4EAA817A05B8307F4E47,src
0,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q8IX94
1,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",S4R3F8
2,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
3,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
4,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
...,...,...
44827,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44828,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44829,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44830,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",A0A096LPK9


In [59]:
disease_id_map = {}
for idx, row in disease_id_df.iterrows():
    v,k = row.to_list()
    disease_id_map[k] = v
    

In [60]:
gene_id_map = {}
for idx, row in gene_id_df.iterrows():
    v,k = row.to_list()
    gene_id_map[k] = v

gene_list = list(set(gene_id_df['src'].to_list()))

In [61]:
edge_df = pd.read_csv("../parser/temp/disease_gene.csv", sep="\t")
edge_df

,src,dst,gene_name,gene_description,pLI,DSI,DPI,source,association_type,sentence,pmid
0,Q9UBX3,C0000727,SLC25A10,solute carrier family 25 member 10,5.872700e-10,0.666,0.500,BEFREE,Biomarker,This rare presentation of acute abdomen and se...,29697421.0
1,O15553,C0000727,MEFV,"MEFV innate immuity regulator, pyrin",1.017700e-14,0.440,0.885,BEFREE,Biomarker,Abdominal FMF attacks resemble the clinical pr...,15833688.0
2,Q7RTS9,C0000729,DYM,dymeclin,3.593900e-13,0.563,0.769,BEFREE,Biomarker,The main action consists in the blockade of Ca...,29732965.0
3,O43291,C0000731,SPINT2,"serine peptidase inhibitor, Kunitz type 2",7.514700e-04,0.556,0.808,HPO,Biomarker,NaN,NaN
4,Q9NYJ7,C0000731,DLL3,delta like canonical Notch ligand 3,5.357600e-05,0.552,0.654,HPO,Biomarker,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2956823,P00846,C4755299,ATP6,ATP synthase F0 subunit 6,NaN,0.541,0.731,ORPHANET,GermlineCausalMutation,Hereditary spastic paraplegia-like disorder du...,20656066.0
2956824,P49368,C4755301,CCT3,chaperonin containing TCP1 subunit 3,9.999800e-01,0.650,0.577,BEFREE,GeneticVariation,"Here, we report a patient with an 8 kb CCTG ex...",25443993.0
2956825,P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,CAPN3 mutations in patients with idiopathic eo...,16607617.0
2956826,P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,Mutations in the gene encoding calpain-3 (CAPN...,17979987.0


In [62]:
edge_df['to_id'] = [disease_id_map[i] for i in edge_df['dst'].to_list()]
edge_df2 = edge_df.copy()
edge_df2 = edge_df2.loc[edge_df2['src'].isin(gene_list)]
edge_df2['from_id'] = [gene_id_map[i] for i in edge_df2['src'].to_list()]
edge_df2

,src,dst,gene_name,gene_description,pLI,DSI,DPI,source,association_type,sentence,pmid,to_id,from_id
0,Q9UBX3,C0000727,SLC25A10,solute carrier family 25 member 10,5.872700e-10,0.666,0.500,BEFREE,Biomarker,This rare presentation of acute abdomen and se...,29697421.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
1,O15553,C0000727,MEFV,"MEFV innate immuity regulator, pyrin",1.017700e-14,0.440,0.885,BEFREE,Biomarker,Abdominal FMF attacks resemble the clinical pr...,15833688.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
2,Q7RTS9,C0000729,DYM,dymeclin,3.593900e-13,0.563,0.769,BEFREE,Biomarker,The main action consists in the blockade of Ca...,29732965.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
3,O43291,C0000731,SPINT2,"serine peptidase inhibitor, Kunitz type 2",7.514700e-04,0.556,0.808,HPO,Biomarker,NaN,NaN,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
4,Q9NYJ7,C0000731,DLL3,delta like canonical Notch ligand 3,5.357600e-05,0.552,0.654,HPO,Biomarker,NaN,NaN,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956823,P00846,C4755299,ATP6,ATP synthase F0 subunit 6,NaN,0.541,0.731,ORPHANET,GermlineCausalMutation,Hereditary spastic paraplegia-like disorder du...,20656066.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
2956824,P49368,C4755301,CCT3,chaperonin containing TCP1 subunit 3,9.999800e-01,0.650,0.577,BEFREE,GeneticVariation,"Here, we report a patient with an 8 kb CCTG ex...",25443993.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
2956825,P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,CAPN3 mutations in patients with idiopathic eo...,16607617.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
2956826,P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,Mutations in the gene encoding calpain-3 (CAPN...,17979987.0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."


In [63]:
edge_df2 = edge_df2[['to_id', 'from_id', 'src', 'dst', 'gene_name', 'gene_description', 'pLI', 'DSI', 'DPI',
       'source', 'association_type', 'sentence', 'pmid']]
edge_df2

,to_id,from_id,src,dst,gene_name,gene_description,pLI,DSI,DPI,source,association_type,sentence,pmid
0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9UBX3,C0000727,SLC25A10,solute carrier family 25 member 10,5.872700e-10,0.666,0.500,BEFREE,Biomarker,This rare presentation of acute abdomen and se...,29697421.0
1,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",O15553,C0000727,MEFV,"MEFV innate immuity regulator, pyrin",1.017700e-14,0.440,0.885,BEFREE,Biomarker,Abdominal FMF attacks resemble the clinical pr...,15833688.0
2,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q7RTS9,C0000729,DYM,dymeclin,3.593900e-13,0.563,0.769,BEFREE,Biomarker,The main action consists in the blockade of Ca...,29732965.0
3,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",O43291,C0000731,SPINT2,"serine peptidase inhibitor, Kunitz type 2",7.514700e-04,0.556,0.808,HPO,Biomarker,NaN,NaN
4,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9NYJ7,C0000731,DLL3,delta like canonical Notch ligand 3,5.357600e-05,0.552,0.654,HPO,Biomarker,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956823,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P00846,C4755299,ATP6,ATP synthase F0 subunit 6,NaN,0.541,0.731,ORPHANET,GermlineCausalMutation,Hereditary spastic paraplegia-like disorder du...,20656066.0
2956824,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P49368,C4755301,CCT3,chaperonin containing TCP1 subunit 3,9.999800e-01,0.650,0.577,BEFREE,GeneticVariation,"Here, we report a patient with an 8 kb CCTG ex...",25443993.0
2956825,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,CAPN3 mutations in patients with idiopathic eo...,16607617.0
2956826,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P20807,C4755301,CAPN3,calpain 3,3.586500e-19,0.564,0.615,BEFREE,GeneticVariation,Mutations in the gene encoding calpain-3 (CAPN...,17979987.0


In [64]:
# DUMP TO DISK
edge_df2.to_csv("./disease_gene_edges.csv", sep="\t", index=False)